In [ ]:
# %pip install datasets sentence-transformers faiss-cpu

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from datasets import load_dataset


embedding = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-nli")

dataset = load_dataset("iPad7/SKN13-3rd-3Team-labeled-datasets-with-first-model", split="train")  
print(dataset[0])  # 확인용 출력

In [ ]:
print(f"데이터 개수: {len(dataset)}")


In [ ]:
### 2. Document 변환 (LangChain용 문서 객체로)
docs = [
    Document(
        page_content=record["리뷰"], 
        metadata={
            "brand": record.get("제품명"),
            "ingredient": record.get("성분"),
            "score": record.get("별점"),
            "skin_type": record.get("피부타입"),
            "concerns": record.get("피부고민"),
            "irritation": record.get("자극도"),
            "category": record.get("카테고리"),
            "emotions": record.get("리뷰_감성"),
        }
    )
    for record in dataset
]

In [ ]:
### 3. 임베딩 모델 설정 (Ko-SBERT)
embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-nli")

### 4. FAISS 벡터DB 생성
vector_db = FAISS.from_documents(docs, embedding_model)
vector_db.save_local("faiss_oliveyoung_reviews")

In [ ]:
# 저장
vector_db.save_local("my_faiss_index")

# 나중에 로드
loaded_db = FAISS.load_local("my_faiss_index", embedding_model)

In [ ]:
query = "지성 피부에 좋은 토너 제품"  # 임의의 쿼리
results = vector_db.similarity_search(query, k=3)

for i, doc in enumerate(results, 1):
    print(f"--- Result {i} ---")
    print("내용:", doc.page_content)
    print("메타데이터:", doc.metadata)
    print()

In [ ]:
# test
results = vector_db.similarity_search("건성 피부에 좋은 수분 제품", k=3)

for i, doc in enumerate(results, 1):
    print(f"--- Result {i} ---")
    print("내용:", doc.page_content)
    print("메타데이터:", doc.metadata)

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
retriever = vector_db.as_retriever()

In [ ]:
# LLM 설정
llm = ChatOpenAI(temperature=0.7, model="gpt-4.1")

# 프롬프트 템플릿
prompt_template = ChatPromptTemplate.from_template("""
      당신은 올리브영 스킨케어 화장품 정보를 전문적으로 안내하는 AI 어시스턴트입니다.
      사용자 질문에 따라 카테고리, 성분, 피부타입, 제형, 자극도, 감정 정보 등을 바탕으로 정확한 화장품 정보를 제공합니다.
      사용자의 피부 고민과 선호에 맞춰 카테고리에 맞는 화장품을 2~3 가지 추천하고, 관련 리뷰 정보를 요약해주는 것이 주요 역할입니다.

      # Instruction(지켜야 할 규칙):
      1. 반드시 제공된 문서(context)의 정보만을 기반으로 답변하세요. 
         문서에 없는 사실을 추측하거나 임의로 생성하지 마세요.
      2. 화장품의 이름과 카테고리는 반드시 '카테고리'와 '제품명' 필드를 참고하여 확인하세요. 
         카테고리는 스킨/토너, 에센스/세럼/앰플, 크림, 로션, 미스트/오일로 구성되어 있습니다.
      3. 가능한 한 명확하고 간결하게 답변하세요.
      4. 화장품 정보를 안내할 때는 다음 순서를 지키세요: 
         - 제품명
         - 주요 성분 (가능한 경우)
         - 평점 또는 긍정 리뷰 요약
      5. 문장 스타일은 전문성과 친근함을 겸비한 대화체로 작성하세요. 
         예: “이 제품은 복합성 피부에 정말 잘 맞는다고 해요!”
      6. “문서에 따르면”, “문맥에서 보면”과 같은 표현은 사용하지 마세요. 자연스럽게 설명만 하세요.
      7. 질문이 모호하거나 정보가 부족할 경우, 필요한 정보를 정중하게 요청하세요.
      8. 출력은 보기 쉽게 줄바꿈을 해서 전달해주세요.
      9. 제품 추천 및 설명 마지막에 특징을 잘 나타내는 문서(별점은 제외)들을 세 개 정도 출력해주세요.
                                                   
      # Context(문서 요약 정보):
      {context}

      # 질문:
      {question}
      """)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

# 예시 질문
question = "건조한 피부에 좋은 크림 추천해줘"
response = rag_chain.invoke(question)

print("🧠 응답:")
print(response)